In [1]:
from PIL import Image
import os
import random
import pandas as pd
from io import BytesIO
import base64
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output, no_update, dash_table
from dash.dependencies import Input, Output


In [2]:
def pil_to_b64(im, enc_format="png", **kwargs):
    """
    Converts a PIL Image into base64 string for HTML displaying
    :param im: PIL Image object
    :param enc_format: The image format for displaying. If saved the image will have that extension.
    :return: base64 encoding
    """

    buff = BytesIO()
    im.save(buff, format=enc_format, **kwargs)
    encoded = base64.b64encode(buff.getvalue()).decode("utf-8")

    return encoded

def get_locations_from_folder_path(folder_path):
    rootdir = folder_path
    locations = []
    for root, subdirs, files in os.walk(rootdir):
        for filename in files:
            file_path = os.path.join(root, filename)
            locations.append(file_path)
    return locations

In [3]:
def create_dash_app(df):
    
    my_list = df.index.tolist()
    app = JupyterDash()
    app.layout = html.Div([
        dcc.Slider(0.1, 2, 0.1,
               value=1,
               id='my-slider'
        ),
        html.H1('Image Dataset from folder Visualization using Dash!'),
        dcc.Dropdown(my_list,my_list[0],id='demo-dropdown_2'),
        html.Div(id="barplot-container_2",children=[]),
        html.Div(id='dd-output-container_2'),
        
        
    ])
    @app.callback(
        Output(component_id="barplot-container_2",component_property="children"),
        [Input(component_id="demo-dropdown_2",component_property="value"),
        Input(component_id='my-slider', component_property='value')]
    )
    def update_output(value,slider_value):
        image = Image.open(df['location'][value])
        image = image.resize((round(image.size[0]*slider_value), round(image.size[1]*slider_value)))

        pil_img = pil_to_b64(image)
        
        children=[
            html.Img(id=f"my-img-{df['location'][value]}",className="image", src="data:image/png;base64, " + pil_img,
                    style={"margin":"25px 50px 75px 100px", "border": "1px solid red","height":"1500","width":"3000"})
                    
                
        ]
        return children
    return app
  
def create_dash_app_from_image_folder(folder_path):
    df = pd.DataFrame()
    df['location'] = get_locations_from_folder_path(folder_path)
    app = create_dash_app(df)
    return app
    

In [4]:
folder_path= r"C:\Users\gprak\Downloads\youtubeVideos\datasets\pokemon\images"
app = create_dash_app_from_image_folder(folder_path)

In [5]:
app.run_server(mode="external",port = 5010)

Dash app running on http://127.0.0.1:5010/
